# Face Detection using Amazon Rekognition

***
This notebook provides a walkthrough of [detect face recognition API](https://docs.aws.amazon.com/rekognition/latest/dg/faces-detect-images.html) in Amazon Rekognition. You can see face details include a bounding box of the face, a confidence value (that the bounding box contains a face), and a fixed set of attributes such as facial landmarks (for example, coordinates of eye and mouth), presence of beard, sunglasses, and so on. 
***

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and draw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir m1tmp
tempFolder = 'm1tmp/'

# Image Face Detection
***

In [ ]:
imagePath = "./static/hat-detection.png"

In [ ]:
img=Image.open(imagePath)
display(img)

#### Call Rekognition to recognize faces in the image

In [ ]:
# Call Amazon Rekognition to recognize faces in the image
with open(imagePath, 'rb') as image:
# Todo-1 Assign rekognition detect moderation labels to detectModerationLabel variable with byte data from image.read()
# resource https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/rekognition.html#Rekognition.Client.detect_faces
    faceDetectsResponse

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Name, Id, Urls and additional information about 
# their facial attributes.

display(faceDetectsResponse)

#### Show image with bounding boxes around recognized faces

In [ ]:
# Define a function that will display image with bounded boxes around recognized faces
# We will call this function in next step
import io

def drawBoundingBoxes (boxes, image):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))

    # Draws BB on Image
    draw = ImageDraw.Draw(image)
    width, height = image.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        left = width * box['Left']
        top = height * box['Top']
        width = width * box['Width']
        height = height * box['Height']
    
        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)
        )
        draw.line(points, fill='#00d400', width=2)
        
    display(image)

In [ ]:
# Extract bounding box information from JSON response above and display image with bounding boxes around faces.

boxes = []
faces = faceDetectsResponse['FaceDetails']
for face in faces:
    boxes.append (face['BoundingBox'])
    
drawBoundingBoxes(boxes,img)

# Try with your own face
- Upload your image to the static folder in this directory
***

In [ ]:
#Todo-2 assign imagePath with your face image
imagePath = ""

In [ ]:
img=Image.open(imagePath)
display(img)

#### Call Rekognition to recognize faces in the image

In [ ]:
# Call Amazon Rekognition to recognize faces in the image
with open(imagePath, 'rb') as image:
    faceDetectsResponse = rekognition.detect_faces(Image={'Bytes': image.read()})

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Name, Id, Urls and additional information about 
# their facial attributes.

display(faceDetectsResponse)

#### Show image with bounding boxes around recognized faces

In [ ]:
# Define a function that will display image with bounded boxes around recognized faces
# We will call this function in next step
import io

def drawBoundingBoxes (boxes, image):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))

    # Draws BB on Image
    draw = ImageDraw.Draw(image)
    width, height = image.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        left = width * box['Left']
        top = height * box['Top']
        width = width * box['Width']
        height = height * box['Height']
    
        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)
        )
        draw.line(points, fill='#00d400', width=2)
        
    display(image)

In [ ]:
# Extract bounding box information from JSON response above and display image with bounding boxes around faces.

boxes = []
faces = faceDetectsResponse['FaceDetails']
for face in faces:
    boxes.append (face['BoundingBox'])
    
drawBoundingBoxes(boxes,img)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/images-bytes.html
- https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/rekognition.html#Rekognition.Client.detect_faces
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectFaces.html

***

# Recognize faces in video
 Face recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceDetection.html. 

- First we start a face detection job which returns a Job Id.
- We can then call `get_face_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

In [ ]:
# Todo-3 download object-detection.mov file and upload to your s3, assign bucketName with your own bucket and videoName with the object key
videoName = ""
bucketName = ""

#### Call Rekognition to start a job for object detection

In [ ]:
# Start video label recognition job
startFaceDetection = rekognition.start_face_detection(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

labelsJobId = startFaceDetection['JobId']
display("Job Id: {0}".format(labelsJobId))

#### Wait for object detection job to complete

In [ ]:
# Wait for object detection job to complete
# In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
getObjectDetection = rekognition.get_face_detection(
    JobId=labelsJobId
)

while(getObjectDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getObjectDetection = rekognition.get_face_detection(
    JobId=labelsJobId)
    
display(getObjectDetection['JobStatus'])

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Object Detection API
# In the JSON response below, you will see list of detected objects and activities.
# For each detected object, you will see information like Timestamp

display(getObjectDetection)

#### Show video in the player

In [ ]:
# Show video in a player

s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))